In [1]:
from astropy.coordinates import get_sun
from astropy.time import Time, TimeDelta
from poliastro.bodies import Earth, Sun
from astropy import units as u
from poliastro.twobody import Orbit

import numpy as np
import trimesh

import numba
import pandas as pd
try:
    from SatelitteSolarPowerSystemV4 import SatelitteSolarPowerSystem
    from SatelitteActitud import SatelitteActitud
    from OrbitasV4 import *
except:
    from src.data.SatelitteSolarPowerSystemV4 import SatelitteSolarPowerSystem
    from src.data.SatelitteActitud import SatelitteActitud
    from src.data.OrbitasV4 import *
from tqdm import tqdm




In [2]:
simulacion = random_generator()

a = 7278 * u.km
ecc = 0 * u.one
inc = 90 * u.deg
raan = 0* u.deg
argp_ini = 0 * u.deg
nu = 0 * u.deg
time = Time("2020-01-01 12:00:00", scale="utc")
i=0
direccion = '12Unuv.stl'
actitud=SatelitteActitud(eje_de_spin= [0,1,0],control=True) 
EPM = SatelitteSolarPowerSystem(direccion, actitud, Despegables_orientables=True)
EPM.apply_transform(trimesh.transformations.rotation_matrix(np.pi/2,[0,0,1],[0,0,0]))

In [73]:
EPM.Caras_Despegables

sat = Orbit.from_classical(Earth, a, ecc, inc, raan, 270*u.deg, nu, time+TimeDelta(70*u.min))
Sun_vector = SolarAngle_a_VNB(sat)

In [74]:
direcion_principal=EPM.mesh.facets_normal[EPM.Caras_Despegables[0]]
            
plano0=np.cross( Sun_vector, EPM.actitud.eje_de_spin)
plano0=plano0/np.linalg.norm(plano0)
print("plano0 ", plano0)
plano1=np.cross( direcion_principal, EPM.actitud.eje_de_spin)
            
plano1=plano1/np.linalg.norm(plano1)
print("plano1 ", plano1)
angulo_giro=np.absolute(np.arccos(np.dot(plano0, plano1))/(np.linalg.norm(plano0)*np.linalg.norm(plano1)))
prim=trimesh.transform_points(plano1.reshape(1,3), trimesh.transformations.rotation_matrix(-angulo_giro,actitud.eje_de_spin,[0,0,0]))
         
if np.isnan(angulo_giro):
    angulo_giro=0.0      
            

            
if angulo_giro==0:
    pass
else:
    prim=trimesh.transform_points(plano1.reshape(1,3), trimesh.transformations.rotation_matrix(angulo_giro,actitud.eje_de_spin,[0,0,0]))
    if not np.allclose(prim, plano0):
        angulo_giro=-angulo_giro
    EPM.mesh=EPM.mesh.apply_transform(trimesh.transformations.rotation_matrix(angulo_giro,actitud.eje_de_spin,[0,0,0]))
                

plano0  [-0.98096483  0.          0.19418549]
plano1  [-0.91749894  0.          0.39773823]


In [75]:
print(angulo_giro)

-0.21362330774755842


In [63]:
np.allclose(plano0, plano1)

True

In [55]:
EPM.mesh=EPM.mesh.apply_transform(trimesh.transformations.rotation_matrix(-angulo_giro,actitud.eje_de_spin,[0,0,0]))

In [51]:
trimesh.transform_points(plano1.reshape(1,3), trimesh.transformations.rotation_matrix(-angulo_giro,actitud.eje_de_spin,[0,0,0]))

array([[-0.92273882,  0.        ,  0.38542582]])

In [58]:
plano0= np.round(plano0, 5)

In [59]:
plano0

array([-0.98107,  0.     , -0.19366])